Data for this project comes directly from the New York State Independent Service Authority (NYISO), which acts as a broker for the state's energy market. They store all their historical load data (on a region-wide basis) as csvs on an FTP server. 

You can read more here: http://www.nyiso.com/public/markets_operations/market_data/load_data/index.jsp

Data was downloaded directly from here: `http://mis.nyiso.com/public/P-58Blist.htm`

Electricity in New York State is generated in 12 'zones', which ahve their own separate markets. See the image below for reference. In this data, the load is split out by zones, as in the 'name' column

<img src='https://business.directenergy.com/~/media/deb/images/callouts/map.ashx?la=en&h=500'>

In [1]:
import zipfile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import urllib
import os
import pandas as pd
import numpy as np

/Users/dyawitz/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Downloading the data
Let's start by pulling all the data for the state for 2001-2015.

The files will come in as zip files, which will go in the `../data/nyiso` folder. 

Then, I'll extract every csv into the `../data/nyiso/all/raw` folder

In [103]:
dates = pd.date_range(pd.to_datetime('2001-01-01'), \
                       pd.to_datetime('2015-12-31'), freq='M')

for date in dates:
    url = 'http://mis.nyiso.com/public/csv/pal/{0}{1}01pal_csv.zip'.format(date.year, str(date.month).zfill(2))
    urllib.urlretrieve(url, "../data/nyiso/{0}".format(url.split('/')[-1]))

def unzip(source_filename, dest_dir):
    with zipfile.ZipFile(source_filename) as zf:
        zf.extractall(dest_dir)

In [109]:
zips = []
for file in os.listdir("../data/nyiso"):
    if file.endswith(".zip"):
        zips.append(file)
for z in zips:
    unzip('../data/nyiso/' + z, '../data/nyiso/all/raw')

Combine all csvs into one file: combined.csv

In [112]:
csvs = []
for file in os.listdir("../data/nyiso/all/raw"):
    if file.endswith("pal.csv"):
        csvs.append(file)

In [114]:
fout=open("../data/nyiso/all/combined.csv","a")

# write the entire first file:
for line in open("../data/nyiso/all/raw/"+csvs[0]):
    fout.write(line)
# now the rest, skipping the headers:    
for file in csvs[1:]:
    f = open("../data/nyiso/all/raw/"+file)
    f.next() # skip the header
    for line in f:
         fout.write(line)
    f.close() # not really needed
fout.close()

## Cleaning and viewing the data

Let's load 14 years of data into a pandas dataframe.

In [115]:
df = pd.read_csv("../data/nyiso/all/combined.csv")

Then, clean out any headers that may be in the data and just take the four columns we're interested in: timestamp, region name, id, and load (where load is electricity demand, in Megawatts). 

In [117]:
cols = df.columns
df.columns = [col.lower().replace(' ', '') for col in cols]
df = df[['timestamp', 'name', 'ptid', 'load']][df.load != 'Load']

Rewrite this data to the csv for later

In [136]:
df.to_csv('../data/nyiso/all/combined.csv', index=False)

In [138]:
df.name.unique()

array(['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'MHK VL',
       'MILLWD', 'N.Y.C._LONGIL', 'NORTH', 'WEST', 'LONGIL', 'N.Y.C.'], dtype=object)

## Create a dictionary of weather stations

We'll need this for later. Now that I have the names of each region as they're represented by the ISO, I took each and looked up the corresponding city and weather station. I put these into a dictionary and pickled it to call in the `03_get_weather_data` notebook.

In [60]:
regions = list(df.name.unique())
region_names = ['Capital', 'Central', 'Dunwoodie', 'Genese', 'Hudson Valley', 'Long Island', 'Mohawk Valley', 'Millwood', 'NYC', 'North', 'West']
cities = ['Albany', 'Syracuse', 'Yonkers', 'Rochester', 'Poughkeepsie', 'NYC', 'Utica', 'Yonkers', 'NYC', 'Plattsburgh', 'Buffalo']
weather_stations = ['kalb', 'ksyr', 'klga', 'kroc', 'kpou', 'kjfk', 'krme', 'klga', 'kjfk', 'kpbg', 'kbuf']

In [64]:
weather_dict = dict(zip(regions, zip(weather_stations, region_names, cities)))
weather_dict

{'CAPITL': ('kalb', 'Capital', 'Albany'),
 'CENTRL': ('ksyr', 'Central', 'Syracuse'),
 'DUNWOD': ('klga', 'Dunwoodie', 'Yonkers'),
 'GENESE': ('kroc', 'Genese', 'Rochester'),
 'HUD VL': ('kpou', 'Hudson Valley', 'Poughkeepsie'),
 'LONGIL': ('kjfk', 'Long Island', 'NYC'),
 'MHK VL': ('krme', 'Mohawk Valley', 'Utica'),
 'MILLWD': ('klga', 'Millwood', 'Yonkers'),
 'N.Y.C.': ('kjfk', 'NYC', 'NYC'),
 'NORTH': ('kpbg', 'North', 'Plattsburgh'),
 'WEST': ('kbuf', 'West', 'Buffalo')}

In [63]:
import joblib
joblib.dump(weather_dict, 'weather_dict.pkl')

['weather_dict.pkl']

## Subset the data

In [ ]:
Take an subset of the data for each zone. A smaller spaital resoluiton will make tying in weather data more accurate.

Plus, it makes the file easier to work with one at a time for testing.

In [77]:
for region in weather_dict.keys():
    subset = df[df.name == region].copy()
    filename = weather_dict[region][1].lower().replace(' ', '') + '.csv'
    subset.to_csv('../data/nyiso/all/' + filename, index=False)

In [172]:
#Here's what one of those would look like:

,timestamp,name,ptid,load
0,2012-01-01 00:00:00,CAPITL,61757,1084.4
1,2012-01-01 00:05:00,CAPITL,61757,1055.3
2,2012-01-01 00:10:00,CAPITL,61757,1056.6
3,2012-01-01 00:15:00,CAPITL,61757,1050.8
4,2012-01-01 00:20:00,CAPITL,61757,1050.8


# Output 2012 data to test on

In [160]:
capital[capital.timestamp < pd.to_datetime('2013-01-01')].to_csv('load2012.csv', index=False)
csvs = []
for file in os.listdir("../data/wunderground/kalb"):
    if file.startswith("2012"):
        csvs.append(file)

fout=open("weather2012.csv","a")

# write the entire first file:
for line in open("../data/wunderground/kalb/"+csvs[0]):
    fout.write(line)
# now the rest, skipping the headers:    
for file in csvs[1:]:
    f = open("../data/wunderground/kalb/"+file)
    f.next() # skip the header
    for line in f:
         fout.write(line)
    f.close() # not really needed
fout.close()

## Download historical forecasts
The NYISO publishes a "day-ahead" forecast. One of the goals of this project is to see if I can outperform that. So I'll download their archived day-ahead forecast for 2014-206

Use the day-ahead forecasts from the NYISO website: http://www.nyiso.com/public/markets_operations/market_data/custom_report/index.jsp?report=load_forecast

In [6]:
nyiso_forecast = pd.read_csv('../data/nyiso_dayahead_forecasts/forecast_2014_2016.csv')

In [7]:
len(nyiso_forecast)

211442

In [11]:
nyiso_forecast.columns = ['timestamp', 'zone', 'forecast', 'gmt']

## Scrap notes (feel free to ignore)

These are a few manual things I tried to compare my model to the forecast for Albany. Consider these cells still under development :)

In [13]:
capital_forecast = nyiso_forecast[nyiso_forecast.zone == 'CAPITL']
capital_forecast.reset_index(inplace=True)
capital_forecast = capital_forecast[['timestamp', 'zone', 'forecast']]
capital_forecast.to_csv('../data/nyiso_dayahead_forecasts/capital_forecast.csv', index=False)

This data was copied in raw from weather.gov. It's the Albany 48 hour temperature foreacst that went into the final pdf presentation. 

In [3]:
forecast_48 = [46,47,47,46,45,43,42,41,41,40,39,38,37,36,36,37,38,40,41,43,44,44,45,45,45,43,41,38,36,34,32,31,30,28,27,26,26,25,25,28,34,39,43,47,51,54,55,56,56,52,49,47,46,45,45,44,42,41,40,39,39,39,39,42,46,50,54,57,60,62,63,64]

In [6]:
phour = pd.to_datetime('1 hour')
start_time = pd.to_datetime('2016-03-28')
[range(1,73)]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72]